<a href="https://colab.research.google.com/github/malraharsh/Pruning-Experiments/blob/master/Pruning_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q tensorflow-model-optimization

In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
import pandas as pd

from tensorflow import keras

# %load_ext tensorboard

# os.mkdir('log')

import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

In [3]:
SHOW = False

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(Train_images, Train_labels), (Test_images, Test_labels) = mnist.load_data()

# pct_data = 0.1
# top = int(np.ceil(Train_images.shape[0] * pct_data))

# (train_images, train_labels), (test_images, test_labels) = (Train_images[:top], Train_labels[:top]), (Test_images[:top], Test_labels[:top])
# print(top)

In [17]:
train_images = train_labels = test_images = test_labels = None

def change_pct_data(pct_data):    
    global train_images, train_labels, test_images, test_labels    
    top = int(np.ceil(Train_images.shape[0] * pct_data))
    (train_images, train_labels), (test_images, test_labels) = (Train_images[:top], Train_labels[:top]), (Test_images[:top], Test_labels[:top])
    print(f"No of data - {top}")

In [ ]:
def train(train_images, train_labels, test_images, test_labels):

    # Normalize the input image so that each pixel value is between 0 to 1.
    train_images = train_images.copy() / 255.0
    test_images = test_images.copy() / 255.0

    # Define the model architecture.
    model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
    ])

    # Train the digit classification model
    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    history = model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
    )

    test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
    history.history['test_loss'] = [test_loss]
    history.history['test_acc'] = [test_accuracy]

    if SHOW:
        # _, baseline_train_accuracy = model.evaluate(
        # train_images, train_labels, verbose=0)

        # print('Baseline train accuracy:', baseline_train_accuracy*100)

        print('Baseline test accuracy:', baseline_test_accuracy*100)

        # print('Baseline difference:', (baseline_train_accuracy - baseline_test_accuracy)*100)

    return model, history.history
    

def prune(train_images, train_labels, model):

    # Compute end step to finish pruning after 2 epochs.
    batch_size = 128
    epochs = 1
    validation_split = 0.1 # 10% of training set will be used for validation set. 

    num_images = train_images.shape[0] * (1 - validation_split)
    end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

    # Define model for pruning.
    pruning_params = {
          'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                   final_sparsity=0.80,
                                                                   begin_step=0,
                                                                   end_step=end_step)
    }

    model_for_pruning = prune_low_magnitude(model, **pruning_params)

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model_for_pruning.summary()

    callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
    #   tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
    ]

    history = model_for_pruning.fit(train_images, train_labels,
                      batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                      callbacks=callbacks)
    
    test_loss, test_accuracy = model_for_pruning.evaluate(test_images, test_labels, verbose=0)
    history.history['test_loss'] = [test_loss]
    history.history['test_acc'] = [test_accuracy]

    if SHOW:
        print('Pruned test accuracy:', model_for_pruning_accuracy)

    return model_for_pruning, history.history

In [ ]:
def do_train(test_pct):
    train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels, test_size=test_pct, stratify=train_labels)
    model, info_train = train(train_x, train_y, val_x, val_y)
    _, info_prune = prune(train_x, train_y, model)
    return info_train, info_prune

In [ ]:
def add_info(dic, pct, info):
    global z
    z=dic
    # print(list(dic.items()), '----')
    dic = {k:v[-1] for k, v in dic.items()}
    dic['percentage'] = pct
    return info.append(dic, ignore_index=True)   

def save(df, name, pct_data):
    df.to_csv(f'log/info-{name}-{pct_data*100}%.csv')

def full_pct_data(pct_data): #pct of full data
    global z
    change_pct_data(pct_data) 
    df_info_train = pd.DataFrame()
    df_info_prune = pd.DataFrame()

    for p in range(10, 99, 90):
        print(f'\n Percentage is {p} \n')

        test_p = 1 - p/100
        info_train, info_prune = do_train(test_p)
        z = info_train

        df_info_train = add_info(info_train, 1 - test_p, df_info_train)
        df_info_prune = add_info(info_prune, 1 - test_p, df_info_prune)

    save(df_info_train, 'train', pct_data)
    save(df_info_prune, 'prune', pct_data)

    # df_info.plot.scatter(x='percentage', y='accuracy')
    # return df_info    

full_pct_data(0.01)

In [ ]:
z

In [79]:
{k:v[-1] for k, v in z.items()}

{'accuracy': 0.09259258955717087,
 'loss': 2.3262572288513184,
 'test_acc': 0.20370370149612427,
 'test_loss': 2.2623438835144043,
 'val_accuracy': 0.1666666716337204,
 'val_loss': 2.432265520095825}